# Orekit in Python - The Basics

Orekit is an open-source library for astrodynamical calculations such as:

* Orbit Propagation
* Coordinate system transformations
* Time systems
* Orbit Determination

And much more..

The orekit webpage can be accessed at http://www.orekit.org.


## The Java - Python bridge - "orekit python wrapper"

The orekit library is written and executing in java. However for interactive there are some advantages using the Python scientific ecosystem, that gives access to a variety of packages ranging from science to visualization and mapping. With the introduction of Jupyter notebooks, this has been highlighted further when a document based approach can be used, in a simple browser interface.

## Architecture 

The architecture of the Orekit_jpype python wrapper is based on the tool jpype, that executes the jvm and provides an interface to the jvm for python. A few helper functions is provided in orekit_jpype for starting up, conversion of format between python and orekit etc. 

There is another "classic" wrapper for orekit based on JCC. If you do not know that you need the JCC version you are very likely better off with the jpype version

The following parts are wrapped:
  - The [Orekit](https://www.orekit.org/) astrodynamics library is explitly wrapped
  - The [Rugged](https://www.orekit.org/rugged/) a sensor to terrain mapping library is also explicitly wrapped
  - The needed [Hipparchus](https://www.hipparchus.org/) mathematics libary is explicitly wrapped
  - Some selected classes from the java tree is also wrapped, classes that are needed for methods or class initialization

A module with some helper files, orekit.pyhelpers, is included for some convertions etc.

The Python version, or correctly stated the Python Wrapper (it is a pure java orekit that runs in background) has a few areas where there are specific ways to do things, limitations and some quirks. The wrapper is using the jpype tool, which has some small quirks and features that may be good to look into, see jpype webpage.

# Installation 

Orekit is a native java library and will run as such, and need a java runtime enviornment (JRE or JDK). A common choice is openjdk. In many cases this is already installed in an operating system, but otherwise google to find a way to install openjdk.

The installation of orekit_jpype can be done through pip.

      pip install orekit_jpype

Or through the newer package manager uv. In this case it is recommended to create a project (like this example repository), and manage the enviroment through uv commands.

To get a fresh virtual python environment and run a jupyter notebook for these examples do run:

        uv run jupyter lab

In the top directory of this repository. All neccessary packages will be installed in a temporary python environment.



### Getting the orekit-data.zip file 

Some fundamental parameters are not included in the orekit package, and should be provided separately. Such files are earth orientation parameters, time standards etc. An example collection of such parameters are available on the orekit page (no guaratees, this data should be maintained by the user for any professional usage).

The datafile is default accessed in current orekit directory or a path can be supplied. The file can be downloaded from https://gitlab.orekit.org/orekit/orekit-data

A convenience function to download this file is included in the orekit package.

## Setting the Envionment parameters

The Orekit wrapper needs a few environment variables to be set. This is automatically done in the conda package when the environment is activated. But it needs to be activated. This can be done through various ways:
  - Starting the jupyter notebook, IDE or other through the Anaconda launcher
  - Using the conda-wrappers tool to get the environment automatically activated when accessing python.bat
  - Manually starting the python from the Anaconda command prompt
  - Hardcoding the JCC_JDK and PATH (on windows) environment parameters in your computer setup
  ...
  
Please see the [wiki](https://gitlab.orekit.org/orekit-labs/python-wrapper/-/wikis/installation) for more details 

# Initiating Orekit in Python

Orekit needs to be initiated before used, this starts up the java engine and exposes the orekit classes in python.

To start up orekit use the initVM() function:

In [17]:
import orekit_jpype, jpype
vm = orekit_jpype.initVM()
print ('JVM location', jpype.getDefaultJVMPath())
jpype.getJVMVersion()
print ('Java version:', jpype.getJVMVersion())


JVM already started, resuming on started JVM
JVM location /usr/lib/jvm/java-21-openjdk-amd64/lib/server/libjvm.so
Java version: (21, 0, 4)


AttributeError: module 'orekit_jpype' has no attribute 'getVersion'

The orekit library needs a data file with various information on time and earth rotation parameters. This file, called orekit-data.zip is loaded from current dir as default. 

This routine of setting up these parameters is commonly used in the notebooks, so a python specific function is created for this, setup_orekit_curdir()

In [2]:
from orekit_jpype.pyhelpers import setup_orekit_data
setup_orekit_data()

Now we are set up to import and use objects from the orekit library. With the proper initialization, all objects in the orekit java library is exposed to Python and can be imported using normal Python syntax.

In [3]:
from org.orekit.utils import Constants

In [4]:
print (Constants.WGS84_EARTH_EQUATORIAL_RADIUS)

6378137.0


SI base units are used in the library, such as seconds, meter, m/s

# Documentation

One of the best sources of knowledge about orekit and the Python wrapper is on the orekit forum, https://forum.orekit.org/

## Orekit API 

The Python version of orekit is very closely matching the java version and the API documentation from Java can be used. This documentation is available at https://www.orekit.org/doc-javadoc.html



In addition to the API documentation there are a number of architecture descriptions, based on the Java syntax but highly applicable to the Python side. See https://www.orekit.org/doc-maven.html


## Usage of parametric types

The Java language has a feature of parametric class definitions and orekit uses this in some areas. These can be managed in Python with the _of(Class) method that sets this.

Example:

## Subclassing Java classes in Python 

In some use cases it is relevant to subclass a java class for an own implementation. Subclassing of Java classes in Python is possible but some adjustments in Java are needed to the classes that are to be subclassed. Instead of modifying the original orekit classes, specific "PythonClassName" classes have been created for interfaces and selected classes.

*It should be noted that all PythonClasses that don't have a test case shall be considered experimental*

Example:

In [5]:
from org.orekit.propagation.sampling import PythonOrekitFixedStepHandler

ImportError: Failed to import 'org.orekit.propagation.sampling.PythonOrekitFixedStepHandler'

In [15]:
class StepHandler(PythonOrekitFixedStepHandler):
    
    def __init__(self, my_value):
        self.my_value = my_value
        super(StepHandler, self).__init__()

    def init(self, s0, t, step):
        pass

    def handleStep(self, state):
        # your code goes here that is executed for every step
        pass

    def finish(self, finalState):
        pass


In the example above, note:

  - The class is subclassing a special class, instead of OrekitFixedStepHandler, PythonOrekitFixedStepHandler is used.
  - All methods in the class / interface needs to be implemented for java to accept it. If not used, just use a pass statement.
  - If a custom __init__ is used, the super needs also to be called to finalize the initialization of the object

## Implementation of Java Interfaces in Python 

The implementation of orekit interfaces in Python is performed in the same way as subclassing, where all interfaces has a corresponding "Python" prefixed version that can be subclassed in python in order to implement that interface.

Example:

Note: It is not possible to implement several interfaces to a Python class

## init or \_\_init\_\_ in subclassing

The Python function for initializing a class is typically the \_\_init\_\_(self, ...) method. Several Orekit interfaces and abstract classes have a init() method that is used for similar purposes. In many Interfaces and Abstract classes, the init(...) method needs to be there for it to be an accepted implementation. For the purpose of populating the interface, it is enough in Python to specify an empty method:

In [ ]:
def init(self):
    pass

In most practical cases when the class is initialized upon creation, it is suitable to use the more Pythonic \_\_init\_\_ method instead. 

## Overloaded Methods / Constructors 

Python does not have overloaded methods / constructors as in Java. For calls to the Java API this is not a problem as the Java side will try to figure out which method / constructor to use. However, when subclassing Java classes in Python that has overloaded methods or constructors, special care is needed.

For these classes and interfaces where Orekit uses overloaded methods (methods with same name but different parameters) special care needs to be taken in the Python implementation to make sure which method that the java side is calling. This can be done by checking the types of the input parameters, or by counting the number of parameters provided. For example:

In [19]:
from org.orekit.attitudes import PythonGroundPointing
from org.orekit.utils import TimeStampedPVCoordinates, TimeStampedFieldPVCoordinates, FieldPVCoordinatesProvider, PVCoordinatesProvider, FieldPVCoordinates, PVCoordinates
from typing import Union

class TestGroundPointing(PythonGroundPointing):
    def getTargetPV(self, pvProv, date, frame) -> Union[TimeStampedPVCoordinates,TimeStampedFieldPVCoordinates]:
        match (pvProv):
            case FieldPVCoordinatesProvider():
                return TimeStampedFieldPVCoordinates(date, FieldPVCoordinates.getZero(date.getField()))
            
            case PVCoordinatesProvider():
                return TimeStampedPVCoordinates(date, PVCoordinates.ZERO)

            case _:
                raise RuntimeError(f'Not supported type of PVCoordinatesProvider: {type(pvProv).__name__}')


See full example in test case at: https://github.com/petrushy/orekit_python_artifacts/blob/version-12.0/test/GroundPointingTest.py

## Performance

The performance of Python Orekit has not been properly measured compared to a pure Java implementation, but there are performance penalties in the interchange between Python and Java. This interaction reduces the possibility of end-to-end Just-In-Time optimizations. But the effect depends a lot on how it is used, a propagation that is fully performed in the Java side and only configured from the Python side will have a minimal impact - the "internal" performance of Orekit is not affected.

Frequent callbacks to Python from Java should be avoided for other purposes than prototyping.

## Pythonic methods 

Orekit is built with the JCC options to translate Java getters and setters to more "Pythonic" class variables. This is performed by detecting .getMethod and .setMethod to simply a .method on the class. Note the change of case as well of first variable. It can be discussed how recommended this is as it provides an addition to the original orekit API. But it can be argued that it does increase readability.

Example:

In [20]:
from org.hipparchus.geometry.euclidean.threed import Vector3D
a = Vector3D(2.0, 0.0, 0.0)
print(a.getX(), a.getY(), a.getZ())
print(a.x, a.y, a.z)

2.0 0.0 0.0
2.0 0.0 0.0


## Variable number of arguments functions

Java and Orekit are for some methods using a variable number of inputs, identified as ... in the API. For example the:

    BooleanDetector.andCombine(EventDetector... detectors) 
    
method in Java. To use this in Python, the variable arguments are placed in a list [], like: 

    BooleanDetector.andCombine([illumination_detector, el_detector])

*Petrus Hyvönen, 2024*